In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import time

In [2]:
rd = pd.read_excel('boost_before_onehot.xlsx')
rd

,unit,func,cat,account,year,month,sales,value,nsto,ncarniv
0,373,store,Sales of Goods,3011,2015,4,-1.331717e+07,-1.351818e+07,1,0
1,373,food,Sales of Goods,3011,2015,4,-1.326249e+06,-1.334552e+06,1,0
2,SO,support,Sales of Goods,3011,2015,4,0.000000e+00,0.000000e+00,1,0
3,CSC,support,Sales of Goods,3011,2015,4,0.000000e+00,0.000000e+00,1,0
4,373,store,Sales of Goods,3061,2015,4,-1.331717e+07,0.000000e+00,1,0
...,...,...,...,...,...,...,...,...,...,...
146693,373,store,Management fees,6435,2020,10,-1.490626e+07,0.000000e+00,4,2
146694,373,food,Management fees,6435,2020,10,-1.301022e+06,0.000000e+00,4,2
146695,SO,support,Management fees,6435,2020,10,0.000000e+00,0.000000e+00,4,0
146696,CSC,support,Management fees,6435,2020,10,0.000000e+00,0.000000e+00,4,0


In [3]:
rd.loc[:,['account', 'year', 'month']] = rd.loc[:,['account', 'year', 'month']].astype(str)

In [4]:
rd.dtypes

unit        object
func        object
cat         object
account     object
year        object
month       object
sales      float64
value      float64
nsto         int64
ncarniv      int64
dtype: object

In [5]:
tmp_df = pd.get_dummies(rd, columns=['unit', 'func', 'cat', 'account', 'year', 'month'], )
tmp_df.shape

(146698, 536)

In [6]:
tmp_df.loc[:,'year_2021'] = 0
tmp_df.year_2021 = tmp_df.year_2021.astype(np.uint8)

In [7]:
y = tmp_df.iloc[:,0]
X = tmp_df.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4, random_state=42)
validX = tmp_df.iloc[500:,1:]
validy = tmp_df.iloc[500:,0]

In [8]:
X.dtypes

value        float64
nsto           int64
ncarniv        int64
unit_373       uint8
unit_522       uint8
              ...   
month_6        uint8
month_7        uint8
month_8        uint8
month_9        uint8
year_2021      uint8
Length: 536, dtype: object

In [10]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dvalid = xgb.DMatrix(validX)

In [11]:
num_boost_round = 999

In [50]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
    # Eval with Mean Absolute Error
    'eval_metric': "rmse",
    #'eval_metric': "mae",
    #'tree_method': 'gpu_hist'
}

In [51]:
time_begin = time.time()
model = xgb.train(
    params,
    dtrain,
    num_boost_round=999,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Training Time: %s seconds" % (str(time.time() - time_begin)))

[0]	Test-rmse:7437901.50000
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:5222634.50000
[2]	Test-rmse:3672204.25000
[3]	Test-rmse:2591145.50000
[4]	Test-rmse:1837518.25000
[5]	Test-rmse:1312930.50000
[6]	Test-rmse:951628.75000
[7]	Test-rmse:708514.37500
[8]	Test-rmse:544798.37500
[9]	Test-rmse:434785.81250
[10]	Test-rmse:361542.96875
[11]	Test-rmse:304864.59375
[12]	Test-rmse:271015.75000
[13]	Test-rmse:242503.10938
[14]	Test-rmse:221069.25000
[15]	Test-rmse:206884.73438
[16]	Test-rmse:187240.90625
[17]	Test-rmse:181261.57812
[18]	Test-rmse:169315.20312
[19]	Test-rmse:163007.92188
[20]	Test-rmse:151476.75000
[21]	Test-rmse:138424.87500
[22]	Test-rmse:132716.43750
[23]	Test-rmse:120536.95312
[24]	Test-rmse:112909.39062
[25]	Test-rmse:104841.42969
[26]	Test-rmse:102201.82812
[27]	Test-rmse:98578.13281
[28]	Test-rmse:93198.39844
[29]	Test-rmse:90271.46875
[30]	Test-rmse:85766.16406
[31]	Test-rmse:76359.00781
[32]	Test-rmse:73445.73438
[33]	Test-rmse:67495.09375
[3

In [15]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
    # Eval with Mean Absolute Error
    'eval_metric': "rmse",
    #'eval_metric': "mae",
    'tree_method': 'gpu_hist',
    'predictor':'gpu_predictor'
}

time_begin = time.time()
model = xgb.train(
    params,
    dtrain,
    num_boost_round=999,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Training Time: %s seconds" % (str(time.time() - time_begin)))

[0]	Test-rmse:7437924.50000
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:5222659.00000
[2]	Test-rmse:3672205.25000
[3]	Test-rmse:2591144.00000
[4]	Test-rmse:1837516.62500
[5]	Test-rmse:1312929.50000
[6]	Test-rmse:951625.12500
[7]	Test-rmse:708514.25000
[8]	Test-rmse:544800.56250
[9]	Test-rmse:434787.78125
[10]	Test-rmse:361543.50000
[11]	Test-rmse:304864.65625
[12]	Test-rmse:271015.37500
[13]	Test-rmse:242503.78125
[14]	Test-rmse:221070.67188
[15]	Test-rmse:206885.12500
[16]	Test-rmse:187241.12500
[17]	Test-rmse:181262.50000
[18]	Test-rmse:169315.71875
[19]	Test-rmse:163008.40625
[20]	Test-rmse:151476.54688
[21]	Test-rmse:138426.00000
[22]	Test-rmse:132717.04688
[23]	Test-rmse:120536.83594
[24]	Test-rmse:112910.07031
[25]	Test-rmse:104841.53125
[26]	Test-rmse:102202.25781
[27]	Test-rmse:98578.37500
[28]	Test-rmse:93198.46875
[29]	Test-rmse:90271.74219
[30]	Test-rmse:85766.17188
[31]	Test-rmse:76358.93750
[32]	Test-rmse:73445.69531
[33]	Test-rmse:67495.32812
[3

In [14]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
#    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,7.479759e+06,9480.492313,7.479702e+06,37807.586062
1,5.253342e+06,6660.526095,5.253823e+06,26858.549083
2,3.695566e+06,4722.891034,3.695851e+06,18277.900665
3,2.608748e+06,3333.115605,2.609143e+06,12931.894753
4,1.851093e+06,2358.376605,1.851772e+06,9546.758332
...,...,...,...,...
994,2.403967e+02,113.202213,2.437102e+03,2312.155982
995,2.402909e+02,113.174527,2.437108e+03,2312.241563
996,2.398525e+02,113.147256,2.436799e+03,2312.091476
997,2.396286e+02,113.031979,2.436669e+03,2311.969483


In [16]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    #for max_depth in range(9,12)
    #for min_child_weight in range(5,8)
    for max_depth in range(7,10)
    for min_child_weight in range(1,4)
]

In [17]:
# Define initial best params and MAE
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    
    time_begin = time.time()
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        #metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
#    mean_mae = cv_results['test-rmse-mean'].min()
#    boost_rounds = cv_results['test-rmse-mean'].idxmin()
#    print("\tRMSE {} for {} rounds".format(mean_mae, boost_rounds))
#    if mean_mae < min_mae:
#        min_mae = mean_mae
#        best_params = (max_depth,min_child_weight)
#print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_mae))


    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].idxmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
    print("Training Time: %s seconds" % (str(time.time() - time_begin)))
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


CV with max_depth=7, min_child_weight=1
	RMSE 415.9849990000001 for 353 rounds
Training Time: 159.13121581077576 seconds
CV with max_depth=7, min_child_weight=2
	RMSE 349.7595124 for 801 rounds
Training Time: 318.496013879776 seconds
CV with max_depth=7, min_child_weight=3
	RMSE 265.90785220000004 for 374 rounds
Training Time: 149.31054186820984 seconds
CV with max_depth=8, min_child_weight=1
	RMSE 131.42516999999998 for 219 rounds
Training Time: 154.20892190933228 seconds
CV with max_depth=8, min_child_weight=2
	RMSE 129.2335132 for 282 rounds
Training Time: 186.54270672798157 seconds
CV with max_depth=8, min_child_weight=3
	RMSE 120.76124220000001 for 355 rounds
Training Time: 214.6001000404358 seconds
CV with max_depth=9, min_child_weight=1
	RMSE 3.9474356 for 131 rounds
Training Time: 111.18423318862915 seconds
CV with max_depth=9, min_child_weight=2
	RMSE 10.348556199999999 for 176 rounds
Training Time: 149.8118555545807 seconds
CV with max_depth=9, min_child_weight=3
	RMSE 11.120

In [18]:
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]
params

{'max_depth': 9,
 'min_child_weight': 1,
 'eta': 0.3,
 'subsample': 1,
 'colsample_bytree': 1,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'tree_method': 'gpu_hist',
 'predictor': 'gpu_predictor'}

In [19]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
#    for subsample in np.linspace(0.5, 1, 5)
#    for colsample in np.linspace(0.5, 1, 5)
]

In [20]:
min_rmse = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    time_begin = time.time()
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best score
#    mean_mae = cv_results['test-rmse-mean'].min()
#    boost_rounds = cv_results['test-rmse-mean'].idxmin()
#    print("\tRMSE {} for {} rounds".format(mean_mae, boost_rounds))
#    if mean_mae < min_mae:
#        min_mae = mean_mae
#        best_params = (subsample,colsample)
#print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_mae))

    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
    print("Training Time: %s seconds" % (str(time.time() - time_begin)))
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

CV with subsample=1.0, colsample=1.0
	RMSE 3.9474356 for 131 rounds
Training Time: 109.01427173614502 seconds
CV with subsample=1.0, colsample=0.9
	RMSE 9373.1816408 for 389 rounds
Training Time: 349.9174168109894 seconds
CV with subsample=1.0, colsample=0.8
	RMSE 11535.6742188 for 237 rounds
Training Time: 205.34576654434204 seconds
CV with subsample=1.0, colsample=0.7
	RMSE 39361.7851566 for 459 rounds
Training Time: 330.23911333084106 seconds
CV with subsample=0.9, colsample=1.0
	RMSE 17.0811676 for 205 rounds
Training Time: 204.87074422836304 seconds
CV with subsample=0.9, colsample=0.9
	RMSE 8790.6228518 for 287 rounds
Training Time: 259.7719531059265 seconds
CV with subsample=0.9, colsample=0.8
	RMSE 10583.735351799998 for 280 rounds
Training Time: 235.14884877204895 seconds
CV with subsample=0.9, colsample=0.7
	RMSE 39902.2734376 for 466 rounds
Training Time: 334.079701423645 seconds
CV with subsample=0.8, colsample=1.0
	RMSE 65.513641 for 186 rounds
Training Time: 173.342995166

In [22]:
params['subsample'] = 1.0
params['colsample_bytree'] = 1.0

In [23]:
min_rmse = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    time_begin = time.time()
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
   
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['rmse'],
            early_stopping_rounds=10)
    
    # Update best score
#    mean_mae = cv_results['test-rmse-mean'].min()
#    boost_rounds = cv_results['test-rmse-mean'].idxmin()
#    print("\tRMSE {} for {} rounds\n".format(mean_mae, boost_rounds))
#   if mean_mae < min_mae:
#        min_mae = mean_mae
#        best_params = eta
#print("Best params: {}, RMSE: {}".format(best_params, min_mae))

    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].idxmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta
    print("Training Time: %s seconds" % (str(time.time() - time_begin)))
print("Best params: {}, RMSE: {}".format(best_params, min_rmse))

CV with eta=0.3
	RMSE 3.9474356 for 131 rounds

Training Time: 126.0732536315918 seconds
CV with eta=0.2
	RMSE 170.73637120000004 for 445 rounds

Training Time: 559.2716689109802 seconds
CV with eta=0.1
	RMSE 53.34963319999999 for 966 rounds

Training Time: 1250.083223104477 seconds
CV with eta=0.05
	RMSE 41.102077200000004 for 998 rounds

Training Time: 1112.3996362686157 seconds
CV with eta=0.01
	RMSE 6997.661035 for 998 rounds

Training Time: 849.7756154537201 seconds
CV with eta=0.005
	RMSE 88803.77968780001 for 998 rounds

Training Time: 876.5434091091156 seconds
Best params: 0.3, RMSE: 3.9474356


In [24]:
params['eta'] = best_params #.1

In [25]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-rmse:7433164.50000
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:5210694.50000
[2]	Test-rmse:3653162.00000
[3]	Test-rmse:2563292.25000
[4]	Test-rmse:1799312.00000
[5]	Test-rmse:1263556.25000
[6]	Test-rmse:888227.06250
[7]	Test-rmse:624929.00000
[8]	Test-rmse:440563.40625
[9]	Test-rmse:311264.31250
[10]	Test-rmse:221349.00000
[11]	Test-rmse:159607.60938
[12]	Test-rmse:116089.21875
[13]	Test-rmse:86623.42969
[14]	Test-rmse:65444.03125
[15]	Test-rmse:52352.25781
[16]	Test-rmse:39232.08203
[17]	Test-rmse:30625.74414
[18]	Test-rmse:25076.47266
[19]	Test-rmse:19911.44922
[20]	Test-rmse:16583.37695
[21]	Test-rmse:14338.87207
[22]	Test-rmse:12614.47949
[23]	Test-rmse:11161.15332
[24]	Test-rmse:9348.08008
[25]	Test-rmse:8089.29639
[26]	Test-rmse:7269.45850
[27]	Test-rmse:6768.87158
[28]	Test-rmse:5945.60498
[29]	Test-rmse:5152.97119
[30]	Test-rmse:4466.70264
[31]	Test-rmse:4176.00391
[32]	Test-rmse:3574.72119
[33]	Test-rmse:3251.51245
[34]	Test-rmse:2901.58472


In [26]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-rmse:7433164.50000
[1]	Test-rmse:5210694.50000
[2]	Test-rmse:3653162.00000
[3]	Test-rmse:2563292.25000
[4]	Test-rmse:1799312.00000
[5]	Test-rmse:1263556.25000
[6]	Test-rmse:888227.06250
[7]	Test-rmse:624929.00000
[8]	Test-rmse:440563.40625
[9]	Test-rmse:311264.31250
[10]	Test-rmse:221349.00000
[11]	Test-rmse:159607.60938
[12]	Test-rmse:116089.21875
[13]	Test-rmse:86623.42969
[14]	Test-rmse:65444.03125
[15]	Test-rmse:52352.25781
[16]	Test-rmse:39232.08203
[17]	Test-rmse:30625.74414
[18]	Test-rmse:25076.47266
[19]	Test-rmse:19911.44922
[20]	Test-rmse:16583.37695
[21]	Test-rmse:14338.87207
[22]	Test-rmse:12614.47949
[23]	Test-rmse:11161.15332
[24]	Test-rmse:9348.08008
[25]	Test-rmse:8089.29639
[26]	Test-rmse:7269.45850
[27]	Test-rmse:6768.87158
[28]	Test-rmse:5945.60498
[29]	Test-rmse:5152.97119
[30]	Test-rmse:4466.70264
[31]	Test-rmse:4176.00391
[32]	Test-rmse:3574.72119
[33]	Test-rmse:3251.51245
[34]	Test-rmse:2901.58472
[35]	Test-rmse:2544.80591
[36]	Test-rmse:2161.53516
[37]	

In [27]:
params

{'max_depth': 9,
 'min_child_weight': 1,
 'eta': 0.3,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'tree_method': 'gpu_hist',
 'predictor': 'gpu_predictor'}

In [28]:
best_model.predict(dvalid)

array([-1.3317166e+07, -1.3262499e+06,  5.1988065e-03, ...,
       -1.9319770e+00, -2.1281097e+00, -6.9750775e+06], dtype=float32)

In [29]:
from sklearn.metrics import mean_squared_error

In [34]:
dvalid.get_label

<bound method DMatrix.get_label of <xgboost.core.DMatrix object at 0x0000024635DA6040>>

In [35]:
mean_squared_error(validy,best_model.predict(dvalid))

1.7459230776397165

In [37]:
validX

,value,nsto,ncarniv,unit_373,unit_522,unit_539,unit_582,unit_602,unit_CSC,unit_SO,...,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,year_2021
500,0.0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
501,0.0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
502,0.0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
503,0.0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
504,0.0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146693,0.0,4,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146694,0.0,4,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146695,0.0,4,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
146696,0.0,4,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
pd.DataFrame([validy,best_model.predict(dvalid)])

,500,501,502,503,504,505,506,507,508,509,...,146688,146689,146690,146691,146692,146693,146694,146695,146696,146697
sales,-1.331717e+07,-1326249.487,0.000000,0.000000,-1.331717e+07,-1326249.487,0.000000,0.000000,-1.331717e+07,-1326249.487,...,-9867086.778,-983551.689,-6975077.024,-1.375842e+07,-1218664.767,-1.490626e+07,-1301022.371,0.0,0.0,-6975077.024
Unnamed 0,-1.331717e+07,-1326249.875,0.005199,0.005199,-1.331717e+07,-1326249.875,0.005199,0.005199,-1.331717e+07,-1326249.875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
best_model.save_model('trial1a.model')
best_model.dump_model('trial1b.model')

### A saved model can be loaded as follows:

```python
bst = xgb.Booster({'nthread': 4})  # init model  
bst.load_model('model.bin')  # load data
```